## 증상분류모델

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 1. 질문을 학습 데이터로 사용하기 위해 텍스트 데이터 벡터화
- 문맥을 반영하기위해 KBERT 모델 사용.

In [2]:
import pandas as pd

In [3]:
# 질문 데이터 불러오기
question_df = pd.read_csv('/content/drive/MyDrive/healthcare project/두통 질문.csv', index_col=0)
question_df

,disease_category,disease_name,intention,question
0,응급질환,두통,예방,두통 예방을 위해 어떤 자세나 활동이 도움이 될까요?
1,응급질환,두통,예방,스트레스를 많이 받는 사람이 두통을 예방하기 위해 어떤 생활습관을 가져야 할까요?
2,응급질환,두통,예방,대상포진 예방접종 후에 어지럼증과 두통이 예방접종과 관련이 있다면 어떻게 되나요?
3,응급질환,두통,예방,갈색세포증과 관련하여 일상생활에서 신경써야할 점은 무엇인가요?
4,응급질환,두통,예방,두통을 예방하기 위한 수면 자세에는 어떤 것들이 있을까요?
...,...,...,...,...
3423,응급질환,두통,치료,두통과 스트레스 반응에 대한 급성 과민성을 진단하고 치료하는 방법에는 어떤 것들이 ...
3424,응급질환,두통,치료,약국에서 구입 가능한 두통 치료제 중에서 어떤 것이 가장 효과적인지 알려주세요.
3425,응급질환,두통,치료,만성 두통을 치료하기 위해 어떤 방법이 있는지 알려주세요.
3426,응급질환,두통,치료,두통의 치료를 위해 어떤 약물이 사용될 수 있을까요?


In [4]:
from sentence_transformers import SentenceTransformer

# KoSentence-BERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/467M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# 벡터화(임베딩)
question_vec = model.encode(question_df['question'].tolist(), batch_size=32)

In [6]:
len(question_vec), question_vec[0]

(3428,
 array([ 2.19613284e-01, -9.01607454e-01,  1.34280711e-01, -3.99087280e-01,
        -6.56399131e-01,  2.89249718e-01, -2.18549475e-01, -1.20221782e+00,
         3.31542462e-01, -2.51379460e-01, -1.19238615e+00, -8.10683250e-01,
        -5.47765374e-01,  4.49536383e-01,  7.57921755e-01, -1.56103706e+00,
         1.55002427e+00,  1.80459964e+00, -1.14636257e-01,  1.78722799e-01,
        -6.56761289e-01,  4.59334999e-02,  6.14282310e-01, -3.36845756e-01,
         3.92549664e-01, -7.37427294e-01,  5.98438382e-01, -7.11544156e-01,
         4.54394892e-02,  1.25989103e+00, -4.30762440e-01,  7.18856812e-01,
        -3.46938550e-01, -6.34605706e-01,  4.70853031e-01,  2.59704143e-01,
        -2.77430594e-01, -1.96138900e-02, -7.47466683e-01, -5.03950655e-01,
        -3.71224910e-01, -1.01627715e-01, -1.15113389e+00, -4.11175877e-01,
        -1.41971648e+00,  2.33880386e-01, -3.18698287e-01,  5.84025085e-01,
         5.56882918e-01, -1.88258793e-02,  1.52258766e+00, -3.60942483e-01,
     

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_intention = le.fit_transform(question_df['intention']) # Series를 2D 배열로

In [8]:
question_df['intention'].unique()

array(['예방', '원인', '정의', '증상', '진단', '치료'], dtype=object)

In [9]:
# 라벨 변환 결과 확인인
le_intention

array([0, 0, 0, ..., 5, 5, 5])

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [17]:
X = question_vec
y = le_intention

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
param_grid = {
    'n_estimators': [150, 200, 250, 300], # 트리의 개수
    'max_depth': [10, 15, 20] # 트리 하나당 분류를 시도하는 질문의 개수
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=13), param_grid, cv=5) # cv=5: 5개의 fold로 k-fold 교차검증
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=13),
             param_grid={'max_depth': [10, 15, 20],
                         'n_estimators': [150, 200, 250, 300]})

In [ ]:
result = pd.DataFrame(grid_search.cv_results_)
result.head(2)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,7.134927,0.085203,0.026170,0.000823,10,150,"{'max_depth': 10, 'n_estimators': 150}",0.825137,0.837887,0.875912,0.810219,0.817518,0.833335,0.023175,12
1,9.518633,0.177767,0.034524,0.000670,10,200,"{'max_depth': 10, 'n_estimators': 200}",0.830601,0.836066,0.874088,0.817518,0.819343,0.835523,0.020483,9


In [ ]:
result[['param_n_estimators', 'params', 'mean_test_score']]

,param_n_estimators,params,mean_test_score
0,150,"{'max_depth': 10, 'n_estimators': 150}",0.833335
1,200,"{'max_depth': 10, 'n_estimators': 200}",0.835523
2,250,"{'max_depth': 10, 'n_estimators': 250}",0.835521
3,300,"{'max_depth': 10, 'n_estimators': 300}",0.835157
4,150,"{'max_depth': 15, 'n_estimators': 150}",0.847920
5,200,"{'max_depth': 15, 'n_estimators': 200}",0.850106
6,250,"{'max_depth': 15, 'n_estimators': 250}",0.848283
7,300,"{'max_depth': 15, 'n_estimators': 300}",0.851203
8,150,"{'max_depth': 20, 'n_estimators': 150}",0.848284
9,200,"{'max_depth': 20, 'n_estimators': 200}",0.847193


In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.858600583090379

In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 57,   2,   0,   1,   2,   3],
       [  0, 110,   1,   5,   2,   1],
       [  0,   7, 101,  11,   7,   0],
       [  0,   6,   8, 144,   6,   1],
       [  0,   5,   2,   6, 114,   3],
       [  3,   1,   1,   5,   8,  63]])

#### 2. 벡터화된 질문의 차원이 크므로 학습시 성능이 좋지않을 수 있어 차원 축소 진행

- 차원축소: pca, 모델: RandomForestClassifier
- 차원을 늘려도 정확도 향상이 거의 없음.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 정규화
question_ss = StandardScaler().fit_transform(question_vec)

# pca
pca = PCA(n_components=50) # 축소할 차원의 개수
question_pca = pca.fit_transform(question_ss)

In [ ]:
pca.explained_variance_ratio_

array([0.11609709, 0.07995649, 0.06058289, 0.05276635, 0.03648964,
       0.03374185, 0.02736324, 0.02351206, 0.02223818, 0.01941289,
       0.01877341, 0.01766712, 0.01621959, 0.0148197 , 0.01443115,
       0.01414343, 0.01276166, 0.01238262, 0.01136667, 0.01116493,
       0.01064634, 0.0099815 , 0.00926424, 0.00912638, 0.00895888,
       0.00823751, 0.00785143, 0.00757601, 0.00737708, 0.00698274,
       0.00666826, 0.00640627, 0.00622595, 0.00617228, 0.00607353,
       0.00573565, 0.00556177, 0.0054169 , 0.00509097, 0.00507568,
       0.00487577, 0.00472585, 0.00460258, 0.00438011, 0.00414433,
       0.00400497, 0.00393255, 0.00381603, 0.00369491, 0.00364251],
      dtype=float32)

In [ ]:
# 축소된 차원이 원본 변동성의 약 80%를 설명함.
a = 0
for n in pca.explained_variance_ratio_:
    a += n
a


0.8021399297285825

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import GridSearchCV

In [ ]:
X = question_pca
y = le_intention

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [ ]:
# clf = RandomForestClassifier(n_estimators=100, random_state=13)

param_grid = {
    'n_estimators': [200, 250, 300], # 트리의 개수
    'max_depth': [10, 15, 20], # 트리 하나당 분류를 시도하는 질문의 개수
    'min_samples_split': [2, 4], # 노드를 나누기 위해 필요한 최소한의 sample 개수
    'min_samples_leaf': [2, 4], # 노드를 나눈 후 생성된 각 노드에 필요한 최소 샘플 수
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=13), param_grid, cv=5) # cv=5: 5개의 fold로 k-fold 교차검증
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=13),
             param_grid={'max_depth': [10, 15, 20], 'min_samples_leaf': [2, 4],
                         'min_samples_split': [2, 4],
                         'n_estimators': [200, 250, 300]})

In [ ]:
y_pred = grid_search.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.858600583090379

In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 57,   2,   0,   0,   2,   4],
       [  0, 107,   2,   7,   2,   1],
       [  0,   7, 100,  10,   9,   0],
       [  0,   8,   6, 144,   7,   0],
       [  0,   5,   2,   5, 116,   2],
       [  3,   0,   1,   2,  10,  65]])

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix

n_components = [50, 100, 150]
best_score = 0
bestpca = None

# 정규화
scaler = StandardScaler()
question_ss = scaler.fit_transform(question_vec)

X = question_ss
y = le_intention
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

for n in n_components:

    # pca
    pca = PCA(n_components = n)
    X_train = pca.fit_transform(X_train_raw)
    X_test = pca.transform(X_test_raw) # X_train에서 학습한 pca 변환을 그대로 사용해야하므로 transform만 사용


    param_grid = {
    'n_estimators': [200, 250, 300], # 트리의 개수
    'max_depth': [10, 15, 20], # 트리 하나당 분류를 시도하는 질문의 개수
    }

    grid_search = GridSearchCV(RandomForestClassifier(random_state=13), param_grid, cv=5) # cv=5: 5개의 fold로 k-fold 교차검증
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    if accuracy > best_score:
        best_score = accuracy
        bestpca = n

print('축소한 차원수:', bestpca, '모델 파라미터:', grid_search.best_params_, '정확도:', best_score)

축소한 차원수: 150 모델 파라미터: {'max_depth': 20, 'n_estimators': 200} 정확도: 0.8760932944606414


In [12]:
confusion_matrix(y_test, y_pred)

array([[ 60,   2,   0,   0,   1,   2],
       [  0, 110,   1,   5,   2,   1],
       [  0,   6, 106,   6,   8,   0],
       [  0,   5,   5, 149,   6,   0],
       [  0,   5,   2,   9, 112,   2],
       [  4,   1,   0,   4,   8,  64]])

- 차원축소: pca, 모델: XGBoost

In [ ]:
pca_xg = PCA(n_components = 150)
X_train = pca_xg.fit_transform(X_train_raw)
X_test = pca_xg.transform(X_test_raw)

In [ ]:
from xgboost import XGBClassifier

param_grid = {
    'n_estimators': [100, 200, 300],  # 트리 개수
    'max_depth': [5, 10, 15],  # 최대 깊이
    'learning_rate': [0.01, 0.05, 0.1]  # 학습률
}

xgb = XGBClassifier(random_state=13)

grid_search = GridSearchCV(xgb, param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...ne,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=13, ...),
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [5, 10, 15],
                         'n_estimators': [100, 200, 300]})

In [ ]:
grid_search.best_params_, grid_search.best_score_

({'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300},
 0.8559424567561459)

In [ ]:
y_pred = grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8702623906705539

- RandomForestClassifier, XGBoost 모델 성능 차이가 거의 없어 코드 실행이 비교적 빨리되는 RandomForestClassifier로 선택

#### 3. 모델 저장 및 재사용

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix

n_components = [50, 100, 150]
best_score = 0
bestpca = None

# 정규화
scaler = StandardScaler()
question_ss = scaler.fit_transform(question_vec)

X = question_ss
y = le_intention
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

for n in n_components:

    # pca
    pca = PCA(n_components = n)
    X_train = pca.fit_transform(X_train_raw)
    X_test = pca.transform(X_test_raw) # X_train에서 학습한 pca 변환을 그대로 사용해야하므로 transform만 사용


    param_grid = {
    'n_estimators': [200, 250, 300], # 트리의 개수
    'max_depth': [10, 15, 20], # 트리 하나당 분류를 시도하는 질문의 개수
    }

    grid_search = GridSearchCV(RandomForestClassifier(random_state=13), param_grid, cv=5) # cv=5: 5개의 fold로 k-fold 교차검증
    grid_search.fit(X_train, y_train)

    y_pred = grid_search.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    if accuracy > best_score:
        best_score = accuracy
        bestpca = n

print('축소한 차원수:', bestpca, '모델 파라미터:', grid_search.best_params_, '정확도:', best_score)

축소한 차원수: 150 모델 파라미터: {'max_depth': 20, 'n_estimators': 300} 정확도: 0.8775510204081632


In [ ]:
grid_search.best_estimator_

RandomForestClassifier(max_depth=20, n_estimators=300, random_state=13)

In [ ]:
import joblib

# 모델 저장
joblib.dump(grid_search.best_estimator_, '/content/drive/MyDrive/healthcare project/headache_randomforest.pkl') #best_estimator_ 는 학습이 완료된 모델 객체이다.
joblib.dump(pca, '/content/drive/MyDrive/healthcare project/headache_pca.pkl')
joblib.dump(scaler, '/content/drive/MyDrive/healthcare project/headache_scaler.pkl')

['/content/drive/MyDrive/healthcare project/headache_scaler.pkl']

In [13]:
import joblib

# 저장된 RandomForestClassifier 모델 불러오기
loaded_model = joblib.load('/content/drive/MyDrive/healthcare project/headache_randomforest.pkl')
loaded_scaler = joblib.load('/content/drive/MyDrive/healthcare project/headache_scaler.pkl')
loaded_pca = joblib.load('/content/drive/MyDrive/healthcare project/headache_pca.pkl')

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


question_ss = loaded_scaler.transform(X_test) # X_test: 위에서 이미 벡터화된 질문
question_pca = loaded_pca.transform(question_ss)

In [23]:
# 예측 수행
y_pred = loaded_model.predict(question_pca)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
accuracy

0.8775510204081632

In [25]:
confusion_matrix(y_test, y_pred)

array([[ 59,   1,   0,   0,   1,   4],
       [  0, 108,   1,   7,   2,   1],
       [  0,   6, 102,  10,   8,   0],
       [  0,   4,   5, 152,   4,   0],
       [  0,   5,   2,   5, 116,   2],
       [  2,   1,   0,   4,   9,  65]])

In [26]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.91      0.94        65
           1       0.86      0.91      0.89       119
           2       0.93      0.81      0.86       126
           3       0.85      0.92      0.89       165
           4       0.83      0.89      0.86       130
           5       0.90      0.80      0.85        81

    accuracy                           0.88       686
   macro avg       0.89      0.87      0.88       686
weighted avg       0.88      0.88      0.88       686



In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 모델 전체 성능 계산
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Overall Precision:", precision)
print("Overall Recall:", recall)
print("Overall F1-score:", f1)

Overall Precision: 0.88084457465621
Overall Recall: 0.8775510204081632
Overall F1-score: 0.8774035460295162
